# 成都地铁站经纬度检索（百度坐标系）

In [ ]:
import pandas as pd
import numpy as np
import requests
import json
from urllib.request import urlopen,quote
import ast
import time

## 成都地铁站名称

In [ ]:
chengdu_subway = ['孵化园', '安仁博物馆小镇', '宽窄巷子', '崔家店', '市二医院', '广福', '广都', '府青路',
 '建设巷', '惠王陵', '成渝立交', '成都东客站', '成都东站', '成都南站', '成都大熊猫繁育研究基地', 
 '成都站', '成都行政学院', '成都西站', '文化宫', '文殊院', '新南门', '春熙路', '昭觉寺南路',
 '李家沱', '杜甫草堂', '来龙', '杨柳河', '槐树店', '武侯大道', '武侯祠', '洪河', '海昌路',               
 '涌泉', '清江西路', '火车北站', '火车南站', '熊猫大道', '牛市口', '牛王庙', '犀浦',
 '狮子山', '玉双路', '玉林路', '环球中心', '理工大学', '琉璃场', '白果林', '百草路',
 '盐市口', '省体育馆', '磨子桥', '神仙树', '红星桥', '红牌楼', '羊犀立交', '耍都',
 '花照壁', '茶店子', '茶店子客运站', '草堂北路', '蜀汉路东', '街子古镇', '衣冠庙', 
 '西南交大', '西南财大', '西岭雪山', '迎宾大道', '迎晖路', '连山坡', '都江堰', 
 '金周路', '金沙博物馆', '金沙遗址博物馆', '金科北路', '金花', '金融城', '锦城广场',
 '锦江宾馆', '锦里', '青城山', '青羊宫', '香香巷', '驷马桥', '骡马市', '高升桥',
 '高新', '黄龙溪古镇', '龙平路', '龙泉驿', '龙爪堰']

## 结果储存器

In [ ]:
station = []
baidu_lng = []
baidu_lat = []
result = []

## 爬虫

In [ ]:
for i, item in enumerate (chengdu_subway):
    address = quote('成都地铁'+item+'站')
    city = quote('成都')
    url = 'http://api.map.baidu.com/geocoding/v3/?address={}&output=json&ak=2G8KFxzYwbtEkLX4cx51RXQlQvn3pXOE&callback=showLocation&city={}'.format(address,city)
    response = urlopen(url)
    answer = response.read().decode()
    result.insert(i, answer)
    station.insert(i, item)
    time.sleep(0.5)
    print ('done for', i, item, len(result))

## 结果示例

'showLocation&&showLocation({"status":0,"result":{"location":{"lng":104.06819111612568,"lat":30.615559309816065},"precise":0,"confidence":60,"comprehension":87,"level":"火车站"}})'

In [ ]:
data = pd.DataFrame({'station': station, 'result':result})

In [ ]:
data.isnull().sum()

## 结果由'showLocation&&showLocation'+字典组成，所以提取结果第26个字符以后的数据。

In [ ]:
len('showLocation&&showLocation')

In [ ]:
data['list'] = data['result'].apply(lambda x: ast.literal_eval(x[26:]))

In [ ]:
data['baidu_lng'] = data['list'].apply(lambda x: x['result']['location']['lng'])

In [ ]:
data['baidu_lat'] = data['list'].apply(lambda x: x['result']['location']['lat'])

# 地理坐标转换

转换成其他坐标系请见： https://www.jianshu.com/p/6e69737cffaa

In [ ]:
import math
x_pi = 3.14159265358979324 * 3000.0 / 180.0
pi = 3.1415926535897932384626  # π
a = 6378245.0  # 长半轴
ee = 0.00669342162296594323  # 偏心率平方

In [ ]:
def bd09_to_gcj02(bd_lon, bd_lat):
    """
    百度坐标系(BD-09)转火星坐标系(GCJ-02)
    百度——>谷歌、高德
    :param bd_lat:百度坐标纬度
    :param bd_lon:百度坐标经度
    :return:转换后的坐标列表形式
    """
    x = bd_lon - 0.0065
    y = bd_lat - 0.006
    z = math.sqrt(x * x + y * y) - 0.00002 * math.sin(y * x_pi)
    theta = math.atan2(y, x) - 0.000003 * math.cos(x * x_pi)
    gg_lng = z * math.cos(theta)
    gg_lat = z * math.sin(theta)
    return [gg_lng, gg_lat]

In [ ]:
h = []
for i in range(0, data.shape[0]):
    h.insert(i, bd09_to_gcj02(data['baidu_lng'][i], data['baidu_lat'][i]))

In [ ]:
data['gd_lng'] = 0.00
data['gd_lat'] = 0.00
for i in range(0, data.shape[0]):
    data['gd_lng'][i] = h[i][0]
    data['gd_lat'][i] = h[i][1]

In [ ]:
data.to_csv('chengdu_subway_gaode.csv', index = False, encoding = 'utf-8-sig')